In [1]:
import ee
import datetime
import os
import itertools
import sys
import re

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import geemap

import subprocess
from subprocess import PIPE

In [2]:
ee.Initialize()

In [3]:
# append upper folder into sys-path during run time so we can
# import our own moduls
sys.path.append('..')

In [4]:
from BackGround_modules.Class_2_Classify_Fourier_Img   import Classification
from BackGround_modules.Class_3_Calculate_the_accuracy import Accuracy_assesment

##### Prepare basic parameters

In [5]:
# define the years to be classified
year_name  = list([f'{span[0]}_{span[1]}' for span in zip(range(1990,2020,3),range(1992,2020,3)) ])[::-1]

# import north_china_plain boundary
North_China_Plain = ee.FeatureCollection("users/wangjinzhulala/North_China_Plain_Python/Boundary_shp/North_China_Plain_Boundary")

In [6]:
year_name

['2017_2019',
 '2014_2016',
 '2011_2013',
 '2008_2010',
 '2005_2007',
 '2002_2004',
 '1999_2001',
 '1996_1998',
 '1993_1995',
 '1990_1992']

##### Prepare input_image for classification

In [19]:
# import Landsat img
Landsat_img = [ee.Image(f"users/wang8052664/Cloud_Free_Img/Landsat_cloud_free_{year}")\
                 .clip(North_China_Plain)  for year in year_name]

# import Fourier img   
Fourier_img = [ee.Image(f"users/Jinzhu_Deakin/North_China_Plain/Fourier_img_harmonic_3/Fourier_img_{year}_harmonic_3")\
                 .clip(North_China_Plain)  for year in year_name[5:]] + \
              [ee.Image(f"users/wang8052664/North_China_Plain/Fourier_imgs/Fourier_img_{year}_harmonic_3")\
                 .clip(North_China_Plain)  for year in year_name[:5]] 

# Import the Index-mean Img
NDVI_img = [ee.Image(f"users/wensomone666/Jinzhu/Mean_NDVI/Year_{year}_Mean_NDVI").rename('Mean_NDVI')  
            for year in year_name ]
NDBI_img = [ee.Image(f"users/wensomone666/Jinzhu/Mean_NDVI/Year_{year}_Mean_NDBI").rename('Mean_NDBI')  
            for year in year_name ]
EVI_img  = [ee.Image(f"users/wensomone666/Jinzhu/Mean_NDVI/Year_{year}_Mean_EVI").rename('Mean_EVI')  
            for year in year_name ]

# stack Landsat_Fourier_Nomalized together
Stack_img = [ee.Image(img) for img in zip(Landsat_img,Fourier_img,NDVI_img,NDBI_img,EVI_img)]

##### Prepare input sample points

In [8]:
path = 'users/Jinzhu_Deakin/North_China_Plain/Sample_with_Landsat_Fourier_Normalized'

# import the Invarient points
Invarient_non_built_sample = [ee.FeatureCollection(f'{path}/Invariant_non_built_{year}_extract_Landsat_Fourier_Normalized_img_2017_2019')\
                                .filterMetadata('Built','equals',0) for year in year_name]

Invarient_built_sample     = [ee.FeatureCollection(f'{path}/Village_point_{year}_extract_Landsat_Fourier_Normalized_img')
                                for year in year_name]
Invarient_sample_merge     = [i[0].merge(i[1]) for i in zip(Invarient_non_built_sample,Invarient_built_sample)]

# import the Each_year points

Each_year_non_built_sample = [ee.FeatureCollection(f'{path}/Verified_point_{year}_extract_Landsat_Fourier_Normalized_img')
                                for year in year_name[:5]]

Each_year_built_sample     = [ee.FeatureCollection(f'{path}/Zone_point_{year}_extract_Landsat_Fourier_Normalized_img')
                                for year in year_name[:5]]

Each_year_sample_merge     = [i[0].merge(i[1]) for i in zip(Each_year_non_built_sample,Each_year_built_sample)]

##### Stack input_img and input_sample together

In [21]:
# first put all inputs together
stack_name   = ['Invarient'] * len(Invarient_sample_merge) + ['Each_year'] * len(Each_year_sample_merge)
stack_year   = year_name + year_name[:5]
stack_img    = Stack_img + Stack_img[:5]
stack_sample = Invarient_sample_merge + Each_year_sample_merge

# stack all ingredients together
Stack_img_sample = [(stack_name[i],stack_year[i],stack_img[i],stack_sample[i]) for i in range(len(stack_name))]

In [22]:
Classificatioin_result = []

for input_variable in Stack_img_sample:
    
    classificaiton_tpye = input_variable[0]
    year = input_variable[1]
    sample_pt = input_variable[3]
    input_img = input_variable[2]
    input_band = input_img.bandNames().getInfo()

    # Instatiate the class with a name.
    classification = Classification(year_name      = year,
                                    Verified_point = sample_pt,
                                    Input_img      = input_img,
                                    Input_band     = input_band,
                                    Tree_num       = 100,
                                    Zone_sample    = None,
                                    classProperty  = 'Built')


    # perform the classification on Input_img and Samples
    classification.Stp_1_Classification_on_img()
    classification.Stp_2_Classification_on_Samples()

    # Get the classified img
    classified_img = classification.classification_img

    # Get the classified samples
    train_sample_classified        = classification.Train_sample_classification
    test_sample_classified         = classification.Test_sample_classification
    
    # add classfication into the list
    Classificatioin_result.append((classificaiton_tpye,year,classified_img,test_sample_classified))

##### Export the result

In [35]:
Classificatioin_result[:4]

[('Invarient',
  '2017_2019',
  <ee.featurecollection.FeatureCollection at 0x25bd5dd1160>),
 ('Invarient',
  '2014_2016',
  <ee.featurecollection.FeatureCollection at 0x25bd5da05c0>),
 ('Invarient',
  '2011_2013',
  <ee.featurecollection.FeatureCollection at 0x25bd5dcd1d0>),
 ('Invarient',
  '2008_2010',
  <ee.featurecollection.FeatureCollection at 0x25bd5da0d30>)]

In [37]:
for item in Classificatioin_result[:4]:
    
    asset_path = 'users/wangjinzhulala/North_China_Plain_Python/classification_img'
    
    Classification_type = item[0]
    year = item[1]
    img = item[2]
    
    export_name = f'{Classification_type}_{year}'
    
    task = ee.batch.Export.image.toAsset(   image        = img,
                                            description  = export_name,
                                            assetId      = f'{asset_path}/export_name',
                                            region       = North_China_Plain.geometry().bounds(),
                                            scale        = 30,
                                            maxPixels    = int(1e13))
    task.start()
    
    print(f'Exporting {export_name} successful!')

Exporting Invarient_2017_2019 successful!
Exporting Invarient_2014_2016 successful!
Exporting Invarient_2011_2013 successful!
Exporting Invarient_2008_2010 successful!
